<a href="https://colab.research.google.com/github/timmmariya/Empirical-Law-Studies/blob/main/4_%D0%A0%D0%B0%D0%B1%D0%BE%D1%87%D0%B0%D1%8F_%D1%80%D0%B0%D1%81%D1%87%D0%B5%D1%82%D0%BA%D0%B0_(%D0%B1%D0%B5%D0%B7_%D0%BA%D0%BE%D0%BC%D0%BC%D0%B5%D0%BD%D1%82%D0%B0%D1%80%D0%B8%D0%B5%D0%B2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Подключение библиотек, установка пакетов

In [ ]:
%%time 

# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 
# Install Folium for Geographic data visualization
!pip install folium
# Install plotlyExpress
!pip install plotly_express

Reading package lists... Done
Building dependency tree       
Reading state information... Done
gdal-bin is already the newest version (2.2.3+dfsg-2).
python-gdal is already the newest version (2.2.3+dfsg-2).
python3-gdal is already the newest version (2.2.3+dfsg-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-rtree is already the newest version (0.8.3+ds-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
  Cloning git://github.com/geopandas/geopandas.git to /tmp/pip-req-build-seaenqf9
  Running command git clone -q git://github.com/geopandas/geopandas.git /tmp/pip-req-build-seaenqf

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from scipy import stats
import locale
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


https://colab.research.google.com/github/shakasom/GDS/blob/master/Part1%20-%20Introduction.ipynb#scrollTo=A91qAwyjOTs8

# Загрузка и настраивание проекции карты России

In [ ]:
regions = gpd.read_file('/content/drive/MyDrive/НУЛ АДП Материалы Марии Тимониной/_data/RegionsFolder/russia_forest_25people.geojson')

https://habr.com/ru/post/459902/

In [ ]:
regions = regions.to_crs('ESRI:102012')

## Сопоставление кодов Арбитражных судов и регионов России из датафрейма карты

# Рассчет матриц проверки гипотезы о средних для заданных дат и уровня значимости

In [ ]:
pathid = "/content/drive/MyDrive/НУЛ АДП Материалы Марии Тимониной/_data/id_regionid.csv"
id_regid = pd.read_csv(pathid, sep=';')

In [ ]:
dict_ids = id_regid.set_index('ID').to_dict()['RID']

In [ ]:
#dict_ids

In [ ]:
path = "/content/drive/MyDrive/НУЛ АДП Материалы Марии Тимониной/_data/twenty_years_base/2015_2020_cleaned_dates.csv"
df = pd.read_csv(path, sep=',')

df['StartDate'] = pd.to_datetime(df['StartDate'])
df['FinishDate'] = pd.to_datetime(df['FinishDate'])

df = df[df['CategoryID'] < 203]

In [ ]:
df.shape[0]

379994

In [ ]:
def K_value(categ1, categ2, df_work) :
    mean1 = df_work.at[categ1, 'mean']
    mean2 = df_work.at[categ2, 'mean']

    std1 = df_work.at[categ1, 'std']
    std2 = df_work.at[categ2, 'std']

    n1 = df_work.at[categ1, 'count']
    n2 = df_work.at[categ2, 'count']

    k = (mean1 - mean2) / np.sqrt(std1 * std1 / n1 + std2 * std2 / n2)
    return k

def norm_distr_quantiles(alpha) :
    norm_rv = stats.norm(loc=0, scale=1)  # задали генератор 
    sample = norm_rv.rvs(1000)  # сгенерируем 1000 значений

    gamma = 1 - alpha / 2

    q_right = norm_rv.ppf(gamma)
    q_left = (-1) * q_right
    return gamma, q_left, q_right

In [ ]:
from datetime import datetime as dt
from datetime import timedelta

courts_met = df.groupby('CourtID')['ConsidTime'].agg(['count']).index.to_list()
courtot = len(courts_met)

def pairwise_hypothesis_test_courts(left_border_date, right_border_date, dest, alpha=0.05, min_count=0, fs=15, df=df) :

    # отбор из полной базы всех дел за период от левой граничной даты до правой
    df_parse = df[(df['StartDate'] <= pd.to_datetime(right_border_date)) & (df['FinishDate'] >= pd.to_datetime(left_border_date))]
    # подсчет средних, стандартных отклонений, кол-ва дел для каждой из категорий -> сохранение в dataframe df_work
    df_work = df_parse.groupby('CourtID')['ConsidTime'].agg(['mean', 'std', 'count'])
    # рассматриваются только состоятельные категории, содержащие не менее min_count дел
    df_work = df_work[df_work['count'] > min_count]

    court_list_used = df_work.index
    cnum = len(court_list_used)

    useid = [0 for i in range(cnum)]
    p = 0
    for i in range(cnum) :
        while (courts_met[p] < court_list_used[i]) :
          p += 1
        useid[i] = p
        p += 1

    # Подсчет значения критерия К для всех пар категорий
    kdata = [[0] * courtot for i in range(courtot)]
    for i in range(cnum) :
        for j in range(cnum) :
            ii = useid[i]
            jj = useid[j]
            kdata[ii][jj] = K_value(court_list_used[i], court_list_used[j], df_work)
    df_kcrit = pd.DataFrame(kdata, index=range(courtot), columns=range(courtot))

    # определение границ доверительного интервала
    gamma, q_left, q_right = norm_distr_quantiles(alpha)
    # проверка гипотезы о равенстве средних для каждой пары
    hy_data = [[2] * courtot for i in range(courtot)]
    for i in range(cnum) :
        for j in range(cnum) :
            ii = useid[i]
            jj = useid[j]
            if (kdata[ii][jj] > q_left) and (kdata[ii][jj] < q_right) :
                hy_data[ii][jj] = 1
            else :
                hy_data[ii][jj] = 0
    df_hypo = pd.DataFrame(hy_data, index=range(courtot), columns=range(courtot))

    fits = [0 for i in range(courtot)]
    for i in range(cnum) :
        sum = 0
        for j in range(cnum) :
            ii = useid[i]
            jj = useid[j]
            sum += hy_data[ii][jj]
        fits[useid[i]] = sum
    dest['fits'] = fits

    #sns.heatmap(df_hypo, square=True, ax=axi, cbar=False)

In [ ]:
def create_regions_map_from_hypo_matrix(start_date, period, q_of_shown=15) :
    lb = pd.to_datetime(start_date)
    rb = pd.to_datetime(lb) + pd.to_timedelta(period)

    tc = pd.DataFrame([courts_met]).transpose()
    tc.rename(columns={0 : 'Court'}, inplace=True)

    pairwise_hypothesis_test_courts(lb, rb, tc, alpha=0.1)

    tc.set_index('Court', inplace=True)
    tc['Region_id'] = tc.index.map(dict_ids)

    tc.sort_values(by='fits', ascending=True, inplace=True)
    df_test = tc[['fits', 'Region_id']].set_index('Region_id')

    def change(x, border) :
      if x in border:
          return 1
      else:
          return 0

    reg_test = regions[['id', 'main_name', 'geometry']]
    reg_test.set_index('id', inplace=True)

    df_joined = reg_test.join(df_test)
    border = df_joined.sort_values(by=['fits', 'main_name'])['main_name'].values[:(q_of_shown - 1)]
    df_joined['show'] = df_joined['main_name'].apply(lambda x: change(x, border))
    print((df_joined['show'] == 1).sum())
    list_show = df_joined[df_joined['show'] == 1].sort_values(by='main_name')['main_name'].values
    str_show = ''
    for word in list_show:
        str_show += word
        str_show += '\n\n'

    fig, ax = plt.subplots(1, figsize=(30, 10))
    ax.axis('off')

    # add a title and annotation
    ax.set_title('Регионы РФ c отличительной средней Продолжительностью рассмотрения дел', fontdict={'fontsize': '20', 'fontweight' : '3'})
    ax.annotate('Источник: Картотека арбитражных дел РФ', xy=(0.6, .05), xycoords='figure fraction', fontsize=12, color='#555555')

    regtext = mpatches.Patch(label=str_show, color='white', linewidth=0)
    plt.legend(handles=[regtext], bbox_to_anchor=(0.97, 0.95), loc='upper left', borderaxespad=0., facecolor='white', fontsize=13)
    ax.annotate(lb.strftime("%B %Y") + ' - '+ rb.strftime("%B %Y"), xy=(0.3, .2), xycoords='figure fraction', fontsize=15, color='k')
    df_joined.plot(column='show', cmap='Blues', linewidth=0.8, ax=ax, edgecolor='0.8')

    plt.tight_layout()
    plt.savefig('/content/drive/MyDrive/НУЛ АДП Материалы Марии Тимониной/_Colab Notebooks НУЛ АДП/pics/' + lb.strftime("%Y-%m-%d") + '.png')
    plt.close(fig)

In [ ]:
from datetime import timedelta
from datetime import datetime as dt

base = 2015

for i in range(5) :
    for j in range(2):
        start_date = dt(base + i, 5*j + 1, 1)
        create_regions_map_from_hypo_matrix(start_date, timedelta(days=364), q_of_shown=15)
        print(start_date)

14
2015-01-01 00:00:00
14
2015-06-01 00:00:00
15
2016-01-01 00:00:00
15
2016-06-01 00:00:00
15
2017-01-01 00:00:00
15
2017-06-01 00:00:00
15
2018-01-01 00:00:00
15
2018-06-01 00:00:00
15
2019-01-01 00:00:00
15
2019-06-01 00:00:00


In [ ]:

#im = Image.open('/content/drive/MyDrive/НУЛ АДП Материалы Марии Тимониной/_Colab Notebooks НУЛ АДП/pics/first_check.png')

#im.crop((100, 75, 300, 150))